### 1. Use Pandas to repeat the Week 5 Exercise 5.7. 

Hint 1: Use `pd.read_csv(url, ...)` to read the USGS online gauge data.

Hint 2: Use `df.set_index()` to set the datetime column and `df.resample()` to calculate the monthly and annual statistics

In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import urllib
from urllib.request import urlopen
import math
import datetime

def generate_url(gagelist, begin_date, end_date):       
    gages = ('&site_no={}' * len(gagelist)).format(*gagelist)
    period = f'&period=&begin_date={begin_date}&end_date={end_date}'
    return f'https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb{gages}&referred_module=sw{period}'

print(generate_url(['06803495', '06803486'], '2000-10-10', '2019-09-30'))

url="https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb&site_no=06803486&referred_module=sw&period=&begin_date=2000-10-10&end_date=2019-09-30"

url95=generate_url(['06803495'], '2000-10-10', '2019-09-30')
url86=generate_url(['06803486'], '2000-10-10', '2019-09-30')

out_data95= pd.DataFrame(columns=['monthly95_max','monthly95_min','monthly95_mean'])

data95 = pd.read_csv(url95, header=27, sep='\t', engine='python').drop(index=0)
data86 = pd.read_csv(url86, header=27, sep='\t', engine='python').drop(index=0)

#Hint 2: Use df.set_index() to set the datetime column and df.resample() to calculate the monthly and annual statistics
data95 = pd.read_csv(url95, header=27, sep='\t', engine='python').drop(index=0)
data95.index=pd.to_datetime(data95['datetime'], format='%Y-%m-%d')
data95['93536_00060_00003']=data95['93536_00060_00003'].astype(float)
monthly95_mean=data95['93536_00060_00003'].resample('M').mean()
monthly95_max=data95['93536_00060_00003'].resample('M').max()
monthly95_min=data95['93536_00060_00003'].resample('M').min()
out_data95_monthly=pd.DataFrame({'monthly95_mean':monthly95_mean,'monthly95_max':monthly95_max,'monthly95_min':monthly95_min})
out_data95_monthly.to_csv('out95monthly.csv', float_format='%.2f')   
yearly95=data95['93536_00060_00003'].resample('Y').mean()*86400/43560
out_data95_annual=pd.DataFrame({'yearly95(acre-ft)':yearly95})
out_data95_annual.to_csv('out95annual.csv', float_format='%.2f')


data86 = pd.read_csv(url86, header=27, sep='\t', engine='python').drop(index=0)
data86.index=pd.to_datetime(data86['datetime'], format='%Y-%m-%d')
data86['93535_00060_00003']=data86['93535_00060_00003'].astype(float)
monthly86_mean=data86['93535_00060_00003'].resample('M').mean()
monthly86_max=data86['93535_00060_00003'].resample('M').max()
monthly86_min=data86['93535_00060_00003'].resample('M').min()
out_data86_monthly=pd.DataFrame({'monthly86_mean':monthly86_mean,'monthly86_max':monthly86_max,'monthly86_min':monthly86_min})
out_data86_monthly.to_csv('out86monthly.csv', float_format='%.2f')   
yearly86=data86['93535_00060_00003'].resample('Y').mean()*86400/43560
out_data86_annual=pd.DataFrame({'yearly86(acre-ft)':yearly86})
out_data86_annual.to_csv('out86annual.csv', float_format='%.2f')


https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb&site_no=06803495&site_no=06803486&referred_module=sw&period=&begin_date=2000-10-10&end_date=2019-09-30


### 2. Use Pandas to 

1. Read data in `users.zip` (using Pandas' the on-the-fly decompression cabability)

2. Identify all the occupations and compare the user numbers between STEM-related occupations and non-STEM occupations.

3. Identify the locations of the users that are programmers and above 35. How many male and female programmers, respectively? Is this ratio the same for the age under 35?

3. Compare the numbers of male and female for each occupations

4. Find the occupations with the youngest and oldest **mean** ages, respectively

5. Based on the first two digits of the zip codes, find the area with the largest number of users 

In [2]:


# exercise 7-2-1
data=pd.read_csv(r'C:\Users\mmaguire2\Downloads\users.zip', sep='|') # change path to zip file loaction
# 7-2-2
occupations=pd.DataFrame(data.occupation.unique())  # unique occupations
dups_occupation = data.pivot_table(index=['occupation'], aggfunc='size') # number of users in occupation
STEM_occupations=dups_occupation['doctor']+dups_occupation['engineer']+dups_occupation['healthcare']+dups_occupation['programmer']+dups_occupation['scientist']+dups_occupation['technician']
STEM_occupations
nonSTEM_occupations = len(data)-STEM_occupations
nonSTEM_occupations


# 7-2-3
#Identify the locations of the users that are programmers and above 35. How many male and female programmers, respectively? Is this ratio the same for the age under 35?
location_programmers_35plus=data.query('age>35').query('occupation=="programmer"')['zip_code']
print('Location of programmers 35 plus\n',location_programmers_35plus)
gender_programmers_35plus=data.query('age>35').query('occupation=="programmer"').groupby('gender').agg({'user_id':np.size})
print('Male/female for 35 plus programmer\n', gender_programmers_35plus)

location_programmers_35under=data.query('age<35').query('occupation=="programmer"')['zip_code']
print('Location of programmers under 35\n',location_programmers_35under)
gender_programmers_35under=data.query('age<35').query('occupation=="programmer"').groupby('gender').agg({'user_id':np.size})
print('Male/female for under 35 programmer\n', gender_programmers_35under)

data['age_group'] = pd.cut(data.age, [0, 30, 100])
gender_age = data.pivot_table(values='age', index='occupation', columns=['gender', 'age_group'], aggfunc=['count'])
gender_age


#7-2-4
occupation_MF = data.pivot_table(values='age', index='occupation', columns='gender', aggfunc=['count'])

#7-2-5

occupation_age = data.pivot_table(values='age', index='occupation', aggfunc=['mean'])
occupation_age

#7-2-6
zipco=data['zip_code'][:][0:2]

data['zip_code2']=data.zip_code.astype(str).str[:2]
zipco=data.groupby('zip_code2').agg({'user_id':np.size})
zipco.sort_values(by=['user_id']).tail(1)

Location of programmers 35 plus
 28     94043
54     01331
81     22902
101    30220
140    36117
143    20910
181    33884
199    93402
262    55346
318    22030
351    55105
378    98117
383    45218
420    55105
498    75006
505    03869
693    06365
776    01810
779    94560
791    12205
829    53066
Name: zip_code, dtype: object
Male/female for 35 plus programmer
         user_id
gender         
F             2
M            19
Location of programmers under 35
 16     06355
44     50233
52     55414
57     52246
113    75013
118    67401
133    80236
159    66215
176    19104
218    43212
221    27502
278    85251
282    55305
284    53713
299    55106
304    94086
403    55108
413    38115
469    10021
474    75230
513    20707
550    55414
594    31909
599    02320
602    47905
605    63044
621    14850
624    20723
644    53211
657    92626
660    98121
670    91919
675    32712
681    55128
697    06906
736    98072
742    92660
794    08610
799    55337
863    63021
867    553

,user_id
zip_code2,
55,76
